## Pulling EIA storage data and population-weighted temperature outcomes
Creates calls from the [Energy Information Administration's API](https://www.eia.gov/opendata/)
and Climate Prediction Center's [FTP Site](ftp://ftp.cpc.ncep.noaa.gov/htdocs/degree_days/weighted/daily_data/)

the EIA API series IDs used in this analysis are listed [here](https://www.eia.gov/opendata/qb.php?category=1709237)

**Import dependencies:**

In [1]:
import requests
import datetime
import json
#import io
#import urllib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

##### **Establish API key and organize a dictionary of API series id's for each storage region:**

In [2]:
eia_api_key = '9e4c8d5761a387405ed003e062d45727'

#create dictionary 
eia_api_series_ids = {
    'l48' : 'NG.NW2_EPG0_SWO_R48_BCF.W',
    'east' : 'NG.NW2_EPG0_SWO_R31_BCF.W',
    'midwest' : 'NG.NW2_EPG0_SWO_R32_BCF.W',
    'mountain' : 'NG.NW2_EPG0_SWO_R34_BCF.W',
    'pacific' : 'NG.NW2_EPG0_SWO_R35_BCF.W',
    'south_central' : 'NG.NW2_EPG0_SWO_R33_BCF.W',
    'salt' : 'NG.NW2_EPG0_SSO_R33_BCF.W',
    'nonsalt': 'NG.NW2_EPG0_SNO_R33_BCF.W'
}
#establishes base url for the api call.
#you just need to combine with the API ID string to complete each call
base_url = f'http://api.eia.gov/series/?api_key={eia_api_key}&series_id='

#establish a funcion that generates API calls
def generate_call(n):
    return json.loads(requests.get(base_url + eia_api_series_ids[n]).text)

#establish a dictionaries that to save API calls and DataFrames
call_dict = {}
df_dict = {}


**Create API calls:**

In [3]:
for n in eia_api_series_ids:
    call_dict[n] = generate_call(n)

**establish a function to generate API calls for each region:**

In [4]:
def generate_df(call):
   return pd.DataFrame(call['series'][0]['data'])

In [5]:
for n in call_dict:
    df_dict[n] = generate_df(call_dict[n])

In [6]:
def format_df(n):
    df_dict[n].columns=['week',f'{n}_inventory']
    df_dict[n]['week'] = pd.to_datetime(df_dict[n]['week'])
    df_dict[n] = df_dict[n].sort_values('week')
    df_dict[n] = df_dict[n].set_index('week')
    df_dict[n][f'{n}_change'] = df_dict[n][f'{n}_inventory'] - df_dict[n][f'{n}_inventory'].shift(1)
    df_dict[n].index = df_dict[n].index - datetime.timedelta(days=1)

In [7]:
for n in df_dict:
    format_df(n)

In [12]:
eia_storage_data = df_dict['l48'].merge(
    df_dict['east'], left_index = True, right_index = True).merge(
    df_dict['midwest'], left_index = True, right_index = True).merge(
    df_dict['mountain'], left_index = True, right_index = True).merge(
    df_dict['pacific'], left_index = True, right_index = True).merge(
    df_dict['south_central'], left_index = True, right_index = True).merge(
    df_dict['salt'], left_index = True, right_index = True).merge(
    df_dict['nonsalt'], left_index = True, right_index = True)

### Weather Data Pull

**Pull HDDs**

In [15]:
list_ = []
for x in range(1981,datetime.datetime.now().year+1):
    df = pd.read_csv(f'ftp://ftp.cpc.ncep.noaa.gov/htdocs/degree_days/weighted/daily_data/{x}/Population.Heating.txt',skiprows = 3, delimiter = '|').T
    df.drop(df.index[0], inplace = True)
    list_.append(df)
CPC_HDDs = pd.concat(list_)
CPC_HDDs.index = pd.to_datetime(CPC_HDDs.index)
CPC_HDDs = CPC_HDDs[9]
CPC_HDDs = pd.DataFrame(CPC_HDDs)
CPC_HDDs.columns = ['HDDs']

**Pull CDDs**

In [16]:
list_ = []
for x in range(1981,datetime.datetime.now().year+1):
    df = pd.read_csv(f'ftp://ftp.cpc.ncep.noaa.gov/htdocs/degree_days/weighted/daily_data/{x}/Population.Cooling.txt',skiprows = 3, delimiter = '|').T
    df.drop(df.index[0], inplace = True)
    list_.append(df)
CPC_CDDs = pd.concat(list_)
CPC_CDDs.index = pd.to_datetime(CPC_CDDs.index)
CPC_CDDs = CPC_CDDs[9]
CPC_CDDs = pd.DataFrame(CPC_CDDs)
CPC_CDDs.columns = ['CDDs']

**create a single DataFrame for the weather data**

...and add a Total Degree Day (TDDs) column

In [17]:
CPC_TDDs = pd.merge(CPC_HDDs,CPC_CDDs, left_index = True, right_index = True)
CPC_TDDs['TDDs'] = CPC_TDDs['HDDs'] + CPC_TDDs['CDDs']

reample temperature data to report sum of degree days for each week ending Thursday

In [18]:
CPC_TDDs_Weekly = CPC_TDDs.resample('W-Thu').sum()

In [19]:
combined_df = pd.merge(eia_storage_data,CPC_TDDs_Weekly, left_index = True, right_index = True)